In [1]:
# Import dependencies 

import pandas as pd
from sqlalchemy import create_engine

## Store CSV in DateFrame 

In [2]:
# Store CSV into Dataframe

csv_file2 = "../ETL_project/Resources/Independence100.csv"
independence_df = pd.read_csv(csv_file2)
# independence_df.set_index("Rank", inplace=True)
independence_df.head()

,Rank,Restaurant,Sales,Average Check,City,State,Meals Served
0,1,Carmine's (Times Square),39080335.0,40,New York,N.Y.,469803.0
1,2,The Boathouse Orlando,35218364.0,43,Orlando,Fla.,820819.0
2,3,Old Ebbitt Grill,29104017.0,33,Washington,D.C.,892830.0
3,4,LAVO Italian Restaurant & Nightclub,26916180.0,90,New York,N.Y.,198500.0
4,5,Bryant Park Grill & Cafe,26900000.0,62,New York,N.Y.,403000.0


In [3]:
# Connecting to Local DB

db_connection = "postgres:postgres@localhost:5432/etl_project"
engine = create_engine(f'postgresql://{db_connection}')

In [4]:
# Confirm tables
engine.table_names()

['independence100', 'top250']

In [5]:
# Store CSV into Dataframe

csv_file3 = "../ETL_project/Resources/Top250.csv"
top_df = pd.read_csv(csv_file3)
# top_df.set_index("Rank", inplace=True)
top_df.head()

,Rank,Restaurant,Content,Sales,YOY_Sales,Units,YOY_Units,Headquarters,Segment_Category
0,1,McDonald's,NaN,40412,4.9%,13846,-0.5%,NaN,Quick Service & Burger
1,2,Starbucks,NaN,21380,8.6%,15049,3.0%,NaN,Quick Service & Coffee Cafe
2,3,Chick-fil-A,While Popeyes got a lot of the chicken buzz in...,11320,13.0%,2470,5.0%,NaN,Quick Service & Chicken
3,4,Taco Bell,NaN,11293,9.0%,6766,2.7%,NaN,Quick Service & Mexican
4,5,Burger King,NaN,10204,2.7%,7346,0.2%,NaN,Quick Service & Burger


## Cleaning data

In [6]:
# Cleaning Independence CSV


new_independence_df = independence_df[["Rank", "Restaurant", "Sales", "City", "State", "Meals Served"]].copy()
# new_independence_df = new_independence_df.rename(columns={"Meals Served": "meals_served"})

new_independence_df = new_independence_df.rename(columns={"Rank": "rank", "Restaurant": "restaurant", "Sales": "sales", "City": "city", "State": "state", "Meals Served": "meals_served"})


new_independence_df.head()

,rank,restaurant,sales,city,state,meals_served
0,1,Carmine's (Times Square),39080335.0,New York,N.Y.,469803.0
1,2,The Boathouse Orlando,35218364.0,Orlando,Fla.,820819.0
2,3,Old Ebbitt Grill,29104017.0,Washington,D.C.,892830.0
3,4,LAVO Italian Restaurant & Nightclub,26916180.0,New York,N.Y.,198500.0
4,5,Bryant Park Grill & Cafe,26900000.0,New York,N.Y.,403000.0


In [7]:
# Cleaning top250 CSV

new_top_df = top_df[["Rank", "Restaurant", "Sales", "Units", "Segment_Category"]].copy()

new_top_df = new_top_df.rename(columns={"Rank": "rank", "Restaurant": "restaurant", "Sales": "sales", "Units": "units",
                                     "Segment_Category": "segment_category"})

new_top_df.head()

,rank,restaurant,sales,units,segment_category
0,1,McDonald's,40412,13846,Quick Service & Burger
1,2,Starbucks,21380,15049,Quick Service & Coffee Cafe
2,3,Chick-fil-A,11320,2470,Quick Service & Chicken
3,4,Taco Bell,11293,6766,Quick Service & Mexican
4,5,Burger King,10204,7346,Quick Service & Burger


## Connect to Local Database

In [8]:
# Connecting to Local DB

db_connection = "postgres:postgres@localhost:5432/etl_project"
engine = create_engine(f'postgresql://{db_connection}')

In [9]:
# Confirm tables
engine.table_names()

['independence100', 'top250']

## Load DataFrames into database


In [10]:
# Independence DF

new_independence_df.to_sql(name="independence100", con=engine, if_exists="append", index=False)

In [11]:
# Top250 DF

new_top_df.to_sql(name="top250", con=engine, if_exists="append", index=False)

## Confirm data has been added 

In [12]:
pd.read_sql_query('select * from top250', con=engine).head()

,rank,restaurant,sales,units,segment_category
0,1,McDonald's,40412,13846,Quick Service & Burger
1,2,Starbucks,21380,15049,Quick Service & Coffee Cafe
2,3,Chick-fil-A,11320,2470,Quick Service & Chicken
3,4,Taco Bell,11293,6766,Quick Service & Mexican
4,5,Burger King,10204,7346,Quick Service & Burger


In [14]:
pd.read_sql_query('select * from independence100', con=engine).head()

,rank,restaurant,sales,city,state,meals_served
0,1,Carmine's (Times Square),39080335,New York,N.Y.,469803
1,2,The Boathouse Orlando,35218364,Orlando,Fla.,820819
2,3,Old Ebbitt Grill,29104017,Washington,D.C.,892830
3,4,LAVO Italian Restaurant & Nightclub,26916180,New York,N.Y.,198500
4,5,Bryant Park Grill & Cafe,26900000,New York,N.Y.,403000
